In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [8]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_uas = '/Volumes/Arthur_disk2/Data/CMIP6/Amon/historical/uas/atl4/'
dir_hist_z23 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/z20/atl3/'
dir_hist_z23_2 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/z20/atl4/'


dir_rcp8_uas = '/Volumes/Arthur_disk2/Data/CMIP6/Amon/ssp585/uas/atl4/'
dir_rcp8_z23 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/z20/atl3/'
dir_rcp8_z23_2 = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/z20/atl4/'

path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")
def is_jja(month):
    return (month >= 6) & (month <= 8)


def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended
path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:

model =  ['ACCESS-CM2',  'ACCESS-ESM1-5', 'BCC-CSM2-MR',  'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3',
  'EC-Earth3-Veg','GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0',  'IPSL-CM6A-LR', 'MIROC6',
  'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3']

# Historical

In [9]:


alpha_bf2_hist_jja = np.ones((len(model)))
r_bf2_hist_jja = np.ones((len(model)))
std_bf2_hist_jja = np.ones((len(model)))
p_value_bf2_hist_jja = np.ones((len(model)))

alpha_bf2_hist_monthly = np.ones((len(model),12))
r_bf2_hist_monthly = np.ones((len(model),12))
std_bf2_hist_monthly = np.ones((len(model),12))
p_value_bf2_hist_monthly = np.ones((len(model),12))





for i in range(0,len(model)):
    print(model[i])
    
    uas = xr.open_dataset(dir_hist_uas+'uas_Amon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl4.nc')
    
    try:

        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
    uas = uas.uas.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa_atl4, _ = Atools.ano_norm_t(uas.mean(dim='lon').mean(dim='lat'))


    
    z23_1 = xr.open_dataset(dir_hist_z23+'z20_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl3.nc')
    z23_2 = xr.open_dataset(dir_hist_z23_2+'z20_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg_atl4.nc')
    print(' ')

    try:

        z23_1 = z23_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
        z23_2 = z23_2.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
    except TypeError:
        z23_1['time'] = z23_1.indexes['time'].to_datetimeindex()
        z23_1 = z23_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
        z23_2['time'] = z23_2.indexes['time'].to_datetimeindex()
        z23_2 = z23_2.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31))) 
        
    thermoslope =     z23_1.mean(dim='lon').mean(dim='lat') - z23_2.mean(dim='lon').mean(dim='lat')
        
    thermoslope_tmp2  = nandetrend(thermoslope.z20)
    thermoslope_tmp2 = thermoslope_tmp2.detrended.load()
    thermoslopea, _ = Atools.ano_norm_t(thermoslope_tmp2)

    thermoslopea_atl3_tmp = thermoslopea
    idx = np.isfinite(thermoslopea_atl3_tmp)
    thermoslopea_atl3 = thermoslopea_atl3_tmp[idx]
    uasa_atl4 = uasa_atl4[np.array(idx)]

    uasa_atl4_jja = uasa_atl4.sel(time=is_jja(uasa_atl4['time.month']))
    thermoslopea_atl3_jja = thermoslopea_atl3.sel(time=is_jja(thermoslopea_atl3['time.month']))

    alpha_bf2_hist_jja[i], _, r_bf2_hist_jja[i], p_value_bf2_hist_jja[i], std_bf2_hist_jja[i] = stats.linregress(
    uasa_atl4_jja[:],thermoslopea_atl3_jja[:])
    
    #for j in range(0,12):
    #    alpha_bf2_hist_monthly[i,j], _, r_bf2_hist_monthly[i,j], p_value_bf2_hist_monthly[i,j], std_bf2_hist_monthly[i,j] = stats.linregress(
    #uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1), thermoslopea_atl3.sel(time=thermoslopea_atl3['time.month'] == j+1))
     

ACCESS-CM2
 


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-ESM1-5
 


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


BCC-CSM2-MR


...

 
MRI-ESM2-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


 
NESM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:44: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


In [10]:
bf2_hist_jja  = xr.Dataset({'alpha': (['product'],alpha_bf2_hist_jja),
                       'r': (['product'],r_bf2_hist_jja),
                       'p_value': (['product'],p_value_bf2_hist_jja),
                       'std': (['product'],std_bf2_hist_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF2',
                                    'long_name': 'Second Bjerknes feedback component',
                                    'units': 'm/(m/s)',
                                    'model': 'CMIP6',
                                    'Scenario': 'Historical',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


In [11]:

bf2_hist_jja.to_netcdf(path_data+'bf2_hist_jja_CMIP6_z20.nc',mode='w')

# rcp85

In [12]:


alpha_bf2_rcp8_jja = np.ones((len(model)))
r_bf2_rcp8_jja = np.ones((len(model)))
std_bf2_rcp8_jja = np.ones((len(model)))
p_value_bf2_rcp8_jja = np.ones((len(model)))

alpha_bf2_rcp8_monthly = np.ones((len(model),12))
r_bf2_rcp8_monthly = np.ones((len(model),12))
std_bf2_rcp8_monthly = np.ones((len(model),12))
p_value_bf2_rcp8_monthly = np.ones((len(model),12))

model_id = []

for i in range(0,len(model)):
    
    
    uas = xr.open_dataset(dir_rcp8_uas+'uas_Amon_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl4.nc')

    try:

        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        uas['time'] = uas.indexes['time'].to_datetimeindex()
        uas = uas.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
    uas = uas.uas.chunk(chunks=None)
    uas = xsf.detrend(uas[:],dim='time',type='linear')
    uas = uas.load()
    uasa_atl4, _ = Atools.ano_norm_t(uas.mean(dim='lon').mean(dim='lat'))


    
    z23_1 = xr.open_dataset(dir_rcp8_z23+'z20_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl3.nc')
    z23_2 = xr.open_dataset(dir_rcp8_z23_2+'z20_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg_atl4.nc')
    
    print(model[i])

    try:

        z23_1 = z23_1.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        z23_2 = z23_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
    except TypeError:
        z23_1['time'] = z23_1.indexes['time'].to_datetimeindex()
        z23_1 = z23_1.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
        z23_2['time'] = z23_2.indexes['time'].to_datetimeindex()
        z23_2 = z23_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31))) 
        
    thermoslope =     z23_1.mean(dim='lon').mean(dim='lat') - z23_2.mean(dim='lon').mean(dim='lat')
        
    thermoslope_tmp2  = nandetrend(thermoslope.z20)
    thermoslope_tmp2 = thermoslope_tmp2.detrended.load()
    thermoslopea, _ = Atools.ano_norm_t(thermoslope_tmp2)

    thermoslopea_atl3_tmp = thermoslopea
    idx = np.isfinite(thermoslopea_atl3_tmp)
    thermoslopea_atl3 = thermoslopea_atl3_tmp[idx]
    uasa_atl4 = uasa_atl4[np.array(idx)]

    uasa_atl4_jja = uasa_atl4.sel(time=is_jja(uasa_atl4['time.month']))
    thermoslopea_atl3_jja = thermoslopea_atl3.sel(time=is_jja(thermoslopea_atl3['time.month']))

    alpha_bf2_rcp8_jja[i], _, r_bf2_rcp8_jja[i], p_value_bf2_rcp8_jja[i], std_bf2_rcp8_jja[i] = stats.linregress(
    uasa_atl4_jja[:],thermoslopea_atl3_jja[:])
    
    
   #for j in range(0,12):
   #     alpha_bf2_rcp8_monthly[i,j], _, r_bf2_rcp8_monthly[i,j], p_value_bf2_rcp8_monthly[i,j], std_bf2_rcp8_monthly[i,j] = stats.linregress(
   #uasa_atl4.sel(time=uasa_atl4['time.month'] == j+1), thermoslopea_atl3.sel(time=thermoslopea_atl3['time.month'] == j+1))
     

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-CM2


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


ACCESS-ESM1-5


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


MRI-ESM2-0


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/xarray/core/dataset.py:2039: FutureWarning: None value for 'chunks' is deprecated. It will raise an error in the future. Use instead '{}'
  category=FutureWarning,


NESM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


In [13]:
bf2_rcp8_jja  = xr.Dataset({'alpha': (['product'],alpha_bf2_rcp8_jja),
                       'r': (['product'],r_bf2_rcp8_jja),
                       'p_value': (['product'],p_value_bf2_rcp8_jja),
                       'std': (['product'],std_bf2_rcp8_jja)}
                       ,coords={'product':model},
                        attrs={'standard_name': 'BF2',
                                    'long_name': 'Second Bjerknes feedback component',
                                    'units': 'm/(m/s)',
                                    'model': 'CMIP6',
                                    'Scenario': 'SSP5 8.5',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})



In [14]:
bf2_rcp8_jja.to_netcdf(path_data+'bf2_ssp5_jja_CMIP6_z20.nc',mode='w')